# Mobile price classification with multilayer perceptron
## PyTorch
Erik Matovič and Jakub Horvat

[Dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?select=train.csv)

### 0. Imports

In [1]:
import numpy as np

### 2. Data transformation